# Resizing a pretrained conv network to a new input size

In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

import cv2

import sys
sys.path.insert(0,"/home/lars/libraries/keras/")
import keras
assert keras.__version__[0] == "2", "we work on version 2 of keras"

Using TensorFlow backend.


In [2]:
from keras.layers import Input
from keras.layers import BatchNormalization, SpatialDropout2D
from keras.layers.pooling import MaxPool2D
from keras.layers.convolutional import Conv2D
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Model, load_model

import keras.backend as K

from keras.callbacks import Callback, ModelCheckpoint

### allow dynamic memory allocation

In [3]:
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)
K.set_session(sess)

### Load the pretrained model

In [4]:
extraction_model = load_model("models/tiny_yolo_voc.h5")

/home/lars/libraries/keras/keras/models.py:245: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [5]:
dim_x = 1088
dim_y = 320

In [6]:
try:
    del extraction_model
except Exception as e:
    pass

extraction_model = load_model("models/tiny_yolo_voc.h5")

in_tensor = Input((dim_x, dim_y, 3))
tensor = in_tensor

for layer in extraction_model.layers[1:]:
    tensor = layer(tensor)
    
new_model = Model(inputs=in_tensor, outputs=tensor)

/home/lars/libraries/keras/keras/models.py:245: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [7]:
new_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1088, 320, 3)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            multiple                  432       
_________________________________________________________________
batch_normalization_1 (Batch multiple                  64        
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    multiple                  0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 multiple                  0         
_________________________________________________________________
conv2d_2 (Conv2D)            multiple                  4608      
_________________________________________________________________
batch_normalization_2 (Batch multiple                  128       
__________

In [8]:
new_model.save("models/tiny_"+str(dim_x)+"_"+str(dim_y)+".h5")